# 🧪 Test Notebook - 100K Samples

Quick test with 100K samples and 2 epochs (~30 min)

---

In [ ]:
# 1. Clone repo
!rm -rf EV_Translate_Modle_NLP_Project
!git clone https://github.com/TranKien2005/EV_Translate_Modle_NLP_Project.git
%cd EV_Translate_Modle_NLP_Project

In [ ]:
# 2. Install deps (skip torch)
!pip install -q datasets sentencepiece sacrebleu google-generativeai python-dotenv tqdm tensorboard seaborn pyyaml

In [ ]:
# 3. Create .env with HF token
HF_TOKEN = "YOUR_HF_TOKEN"  # <<< REPLACE THIS!

with open('.env', 'w') as f:
    f.write(f'HF_TOKEN={HF_TOKEN}\n')
print('✓ .env created')

In [ ]:
# 4. Download PhoMT
!python scripts/download_phomt.py

In [ ]:
# 5. Verify download & check paths
from src.config import load_config
config = load_config()

print(f"data_dir: {config.paths.data_dir}")
!ls -la {config.paths.data_dir}/PhoMT/detokenization/ 2>/dev/null || echo "Path not found!"

In [ ]:
# 6. Preprocess (100K only)
!python scripts/preprocess_data.py --max-samples 100000

In [ ]:
# 7. Setup for quick training
import yaml

with open('config/config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

cfg['data']['source'] = 'processed'
cfg['training']['epochs'] = 2

with open('config/config.yaml', 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False)

print('✓ Config: 2 epochs, processed data')

In [ ]:
# 8. Train!
from src.train import Trainer

trainer = Trainer()
trainer.setup()
trainer.train()

In [ ]:
# 9. Test translations
from src.evaluate import load_translator

translator = load_translator(
    'checkpoints/best_model.pt',
    'checkpoints/tokenizers/tokenizer_src.model',
    'checkpoints/tokenizers/tokenizer_tgt.model',
    'config/config.yaml'
)

for s in ["Hello", "How are you?", "Thank you"]:
    print(f"EN: {s} -> VI: {translator.translate(s)}")

## ✅ If you see translations, it works!

Use `train_kaggle.ipynb` for full training.